In [700]:
import simpy
import random
import math
import numpy

In [701]:
RANDOM_SEED = 9782


INTERARRIVAL_MEAN = 6 
INTERARRIVAL_RATE = 1.0 / INTERARRIVAL_MEAN

CALL_MEAN = 5 
CALL_RATE = 1.0 / CALL_MEAN


SERVICE_TIME_OPERATOR1_MEAN = 12
SERVICE_TIME_OPERATOR_VARIENCE = 36
mu= math.log(SERVICE_TIME_OPERATOR1_MEAN**2/math.sqrt(SERVICE_TIME_OPERATOR_VARIENCE+SERVICE_TIME_OPERATOR1_MEAN**2))
sigma=math.sqrt(math.log(1+SERVICE_TIME_OPERATOR_VARIENCE /SERVICE_TIME_OPERATOR1_MEAN))


SERVICE_TIME_OPERATOR2_MEAN = 4
SERVICE_TIME_OPERATOR2_DEVIATION = math.sqrt(3)

BREAK_TIME = 3
BREAK_TIME_RATE = 1/60


CUSTOMER_COUNT = 1000
random.seed(RANDOM_SEED)

In [702]:
# Duration of the conversation between the customer and the operator (Operator1 service time)
service_times_operator1 = []

# Duration of the conversation between the customer and the operator2 (operator2 service time)
service_times_operator2 = []

# Duration of waiting time of the customer between automatic operator and op1 or op2
queue_waiting = []



# The number of customer in the answering system
queue_w_customer = []

# The number of customer in the answering system
queue_w_operator1 = []

# The number of customer in the answering system
queue_w_operator2 = []

#Average number of people waiting to be served by operator1.
queue_w_op1 = []

#Average number of people waiting to be served by operator2.
queue_w_op2 = []

#Average number of customers leaving the system unsatisfied either due to
#incorrect routing or due to long waiting times.
unsatisfied_customer=[]

#Utilization of the answering system.
answering_util=[]


# Simulation end time
end_time = 0;



In [703]:
class Customer(object):
    def __init__(self, name, env, operator1, operator2):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        self.queue_t=0
    def call(self):
        global end_time 
        
        print('%s initiated a call at %g' % (self.name, self.env.now))
        queue_w_customer.append(self.env.now - self.arrival_t)
        prob=random.uniform(0,10)
        error=random.uniform(0,10)
     
        #The answering system can serve 100 callers simultaneously 
        if len(queue_w_customer)<101:
            yield env.timeout(random.expovariate(CALL_RATE))
            answering_util.append(self.env.now-self.arrival_t)
            queue_w_customer.pop()
            self.queue_t=self.env.now
            
            #The voice recognition system is not perfect and makes a mistake with a 0.1 probability
            if error>1 :
                #operator one with probability 0.3 or to operator 2 with probability 0.7.
                if prob<3:
                    queue_w_operator1.append(self.env.now - self.arrival_t)
                    yield self.env.process(self.call1())
                if prob>=3:  
                    queue_w_operator2.append(self.env.now - self.arrival_t)
                    yield self.env.process(self.call2())        
            else:
                #An arriving customer that has been waiting on hold for 10 minutes hangs up and leaves the system (reneging).
                unsatisfied_customer.append(1)
                end_time = self.env.now
                print('%s A caller hangs up -error- at %g' % (self.name, self.env.now))
        else:
            
            print('%s A caller drops without answering.-100 parallel channels- at %g' % (self.name, self.env.now))
            queue_w_customer.pop()


        #arrival to operator1
    def call1(self):
        
        with operator1.request() as req:
            yield req
            if((self.env.now - self.queue_t))<10:
                queue_waiting.append(self.env.now - self.queue_t)
                queue_w_op1.append(1)
                print('%s is assigned to a operator1 at %g' % (self.name, self.env.now))
                
                yield self.env.process(self.get_help1_operator1())
                print('%s is done operator1 at %g' % (self.name, self.env.now))
                end_time = self.env.now
            else:
                #unsatisfied due to long waiting times.
                unsatisfied_customer.append(1)
                queue_waiting.append(10)
                queue_w_op1.append(1)
                print('%s A caller hangs up-10 min- at %g' % (self.name, (self.queue_t+10)))
                
                end_time = self.queue_t+10
            queue_w_operator1.pop()

    #arrival to operator2
    def call2(self):
        
        with operator2.request() as req:
            yield req

            if((self.env.now - self.queue_t))<10:
                queue_waiting.append(self.env.now - self.queue_t)
                queue_w_op2.append(1)
                print('%s is assigned to a operator2 at %g' % (self.name, self.env.now))
                yield self.env.process(self.get_help2_operator2())
                print('%s is done operator2 at %g' % (self.name, self.env.now))
                end_time = self.env.now
            else:
                 #unsatisfied due to long waiting times.
                unsatisfied_customer.append(1)
                queue_waiting.append(10)
                queue_w_op2.append(1)
                print('%s A caller hangs up-10 min- at %g' % (self.name, (self.queue_t+10)))
                end_time = self.queue_t+10
            queue_w_operator2.pop()

      
            

    def get_help1_operator1(self):
        duration = numpy.random.lognormal(mu, sigma)
        while duration < 0:
            duration = numpy.random.lognormal(mu, sigma)

        yield self.env.timeout(duration)
        service_times_operator1.append(duration)
        
    def get_help2_operator2(self):
        duration = random.gauss(SERVICE_TIME_OPERATOR2_MEAN, SERVICE_TIME_OPERATOR2_DEVIATION)
        while duration < 0:
            duration = random.gauss(SERVICE_TIME_OPERATOR2_MEAN, SERVICE_TIME_OPERATOR2_DEVIATION)
            
        yield self.env.timeout(duration)
        service_times_operator2.append(duration)



In [704]:
# Generate new customer that call to call center.
def customer_generator(env, operator1, operator2):
   
    for i in range(CUSTOMER_COUNT):
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE))
        customer = Customer('Customer %s' %(i+1), env, operator1, operator2)
        

In [705]:

#   Generate new break for operator1.
def break_generator1(env, operator1):

    BREAK_TIME_COUNTER1 = 0
    
    while True:
      
        yield env.timeout(random.expovariate(BREAK_TIME_RATE))
        # waits until completing all the customers already waiting for her/him.
        while True:
            with operator1.request() as req:
                yield req
                
                if queue_w_operator1:
                    continue
                else:
                    print('Operator1 gives break at %g' % (env.now))
                    yield env.timeout(BREAK_TIME)
                    print('Operator1 break done at %g' % (env.now))
                    BREAK_TIME_COUNTER1+=1
                    print('break1',BREAK_TIME_COUNTER1)
                    break

      

In [706]:
def break_generator2(env, operator2):
    """Generate new break for operator2."""
    
    BREAK_TIME_COUNTER2 = 0
    
    while True:
        
           
        yield env.timeout(random.expovariate(BREAK_TIME_RATE))
        
        # waits until completing all the customers already waiting for her/him.
        while True:
            with operator2.request() as req:
                yield req
                if queue_w_operator2:
                    continue
                else:
                    print('Operator2 gives break at %g' % (env.now))
                    yield env.timeout(BREAK_TIME)
                    print('Operator2 break done at %g' % (env.now))
                    BREAK_TIME_COUNTER2+=1
                    print('break2',BREAK_TIME_COUNTER2)
                    break

In [707]:
#variables of statistics

#sum of 10 1000 customer data utilization of operator1 
sumUtil1KOP1=0
#sum of 10 1000 customer data utilization of operator2 
sumUtil1KOP2=0

#sum of waiting times of 1k customer
sumOf1kWaitingTimes=0

#Maximum Total Waiting Time to Total System Time Ratio,
maxRatio1K=0

#Sum of operater1 waiting times
sumof1kOp1wTimes=0

#Sum of operater2 waiting times
sumof1kOp2wTimes=0

#Sum of Unsatisfied costumer leaving times
sumof1kUnwTimes=0

#Sum of Util time answering system
sumof1kUtilAnswering=0



#sum of 10 5000 customer data utilization of operator1 
sumUtil5KOP1=0
#sum of 10 5000 customer data utilization of operator2 
sumUtil5KOP2=0

#sum of waiting times of 5k customer
sumOf5kWaitingTimes=0

#Maximum Total Waiting Time to Total System Time Ratio,
maxRatio5K=0

#Sum of operater1 waiting times
sumof5kOp1wTimes=0

#Sum of operater2 waiting times
sumof5kOp2wTimes=0

#Sum of Unsatisfied costumer leaving times
sumof5kUnwTimes=0

#Sum of Util time answering system
sumof5kUtilAnswering=0



In [708]:
#1k
for i in range(10):
    RANDOM_SEED+=1
    env = simpy.Environment()
    operator1 = simpy.Resource(env, capacity = 1)
    operator2 = simpy.Resource(env, capacity = 1)
    env.process(customer_generator(env, operator1, operator2))
    env.process(break_generator1(env, operator1))
    env.process(break_generator2(env, operator2))
    env.run(until=7000)
    
    #statistic calculations
    
    
    print(end_time)
    
    #utilization of operators
    serviceTimeOfOP1=sum(service_times_operator1)
    utilizaztionOfOP1=serviceTimeOfOP1/end_time
    sumUtil1KOP1+=utilizaztionOfOP1
    serviceTimeOfOP2=sum(service_times_operator2)
    utilizaztionOfOP2=serviceTimeOfOP2/end_time
    sumUtil1KOP2+=utilizaztionOfOP2
    
    #total waiting time
    sumOf1kWaitingTimes+=sum(queue_waiting)
    
    #Sum of waiting time of operator1
    sumof1kOp1wTimes+=(sum(queue_w_op1))
    
    #Sum of waiting time of operator2
    sumof1kOp2wTimes+=(sum(queue_w_op2))
    
    #Sum of Unsatisfied costumer leaving times
    sumof1kUnwTimes+=(sum(unsatisfied_customer))
    
    #Sum of Util time answering system

    sumof1kUtilAnswering+=(sum(answering_util)/end_time)


    
    #Max Ratio
    maxRatio1K=max(sum(queue_waiting)/end_time,maxRatio1K)

    
    #reset variables
    service_times_operator1 = []

    service_times_operator2 = []
    
    queue_waiting = []
    
    queue_w_op1 = []
    
    queue_w_op2 = []

    queue_w_customer = []

    queue_w_operator1 = []

    queue_w_operator2 = []
    
    end_time = 0
    
    unsatisfied_customer= []
    
    answering_util=[]
    
    
    
    

Customer 1 initiated a call at 3.01324
Customer 2 initiated a call at 3.5674
Customer 3 initiated a call at 3.92895
Customer 4 initiated a call at 3.98808
Customer 2 is assigned to a operator2 at 4.98243
Customer 3 A caller hangs up -error- at 7.64086
Customer 5 initiated a call at 7.71364
Customer 2 is done operator2 at 9.63242
Customer 1 is assigned to a operator2 at 9.63242
Customer 1 is done operator2 at 14.4318
Customer 4 is assigned to a operator2 at 14.4318
Customer 4 is done operator2 at 21.3165
Customer 5 is assigned to a operator2 at 21.3165
Customer 5 is done operator2 at 24.6464
Customer 6 initiated a call at 24.9806
Customer 7 initiated a call at 25.2417
Customer 8 initiated a call at 29.1295
Customer 7 is assigned to a operator2 at 29.5136
Customer 7 is done operator2 at 33.9539
Customer 6 is assigned to a operator2 at 33.9539
Operator1 gives break at 34.3972
Customer 8 A caller hangs up -error- at 36.8438
Customer 9 initiated a call at 37.1984
Operator1 break done at 37.

Customer 517 initiated a call at 3032.95
Customer 516 is done operator2 at 3035.12
Customer 518 initiated a call at 3042.88
Customer 517 is assigned to a operator2 at 3043.58
Customer 519 initiated a call at 3045.47
Customer 517 is done operator2 at 3049.49
Customer 518 is assigned to a operator2 at 3049.49
Customer 520 initiated a call at 3050.2
Customer 521 initiated a call at 3052.61
Customer 521 is assigned to a operator1 at 3054.86
Customer 518 is done operator2 at 3055.26
Customer 519 is assigned to a operator2 at 3055.26
Customer 519 is done operator2 at 3057.79
Customer 520 is assigned to a operator2 at 3057.79
Customer 521 is done operator1 at 3060.41
Customer 520 is done operator2 at 3061.86
Customer 522 initiated a call at 3065.04
Customer 523 initiated a call at 3065.59
Customer 522 is assigned to a operator2 at 3070.95
Customer 524 initiated a call at 3071.43
Customer 523 A caller hangs up -error- at 3071.87
Customer 522 is done operator2 at 3074.49
Customer 524 is assigne

Operator1 break done at 5632.54
break1 77
Customer 951 is done operator2 at 5635.06
Customer 952 initiated a call at 5636.31
Customer 952 is assigned to a operator1 at 5643.04
Customer 953 initiated a call at 5643.59
Customer 954 initiated a call at 5643.89
Customer 953 is assigned to a operator2 at 5646.18
Customer 953 is done operator2 at 5650.25
Customer 955 initiated a call at 5664.76
Customer 956 initiated a call at 5665.47
Customer 955 is assigned to a operator2 at 5665.85
Customer 952 is done operator1 at 5665.91
Customer 954 A caller hangs up-10 min- at 5654.73
Operator1 gives break at 5666.93
Customer 957 initiated a call at 5667.74
Customer 955 is done operator2 at 5668.49
Operator1 break done at 5669.93
break1 78
Customer 957 is assigned to a operator2 at 5671.24
Operator1 gives break at 5673.68
Customer 957 is done operator2 at 5674.35
Operator1 break done at 5676.68
break1 79
Customer 956 A caller hangs up -error- at 5682.66
Customer 958 initiated a call at 5692.06
Custome

Customer 352 initiated a call at 2120.68
Customer 345 is done operator2 at 2121.95
Customer 351 is assigned to a operator2 at 2121.95
Customer 351 is done operator2 at 2127.68
Operator2 gives break at 2127.68
Operator2 break done at 2130.68
break2 32
Customer 352 is assigned to a operator2 at 2130.68
Customer 352 is done operator2 at 2132.7
Customer 338 is done operator1 at 2134.31
Customer 340 A caller hangs up-10 min- at 2082.34
Customer 347 A caller hangs up-10 min- at 2118.23
Customer 348 A caller hangs up-10 min- at 2122.37
Customer 353 initiated a call at 2135.3
Customer 353 is assigned to a operator2 at 2141.14
Customer 354 initiated a call at 2142.97
Customer 353 is done operator2 at 2145.96
Customer 354 is assigned to a operator1 at 2146.93
Customer 355 initiated a call at 2147.61
Customer 355 is assigned to a operator2 at 2149.82
Customer 354 is done operator1 at 2150.65
Customer 355 is done operator2 at 2151.98
Operator1 gives break at 2155.47
Customer 356 initiated a call a

Customer 797 is assigned to a operator1 at 4982.14
Customer 797 is done operator1 at 4982.53
Operator1 gives break at 4982.53
Customer 800 initiated a call at 4984.55
Customer 801 initiated a call at 4985.1
Customer 802 initiated a call at 4985.43
Operator1 break done at 4985.53
break1 61
Customer 800 is assigned to a operator1 at 4985.72
Customer 801 is assigned to a operator2 at 4985.84
Customer 803 initiated a call at 4989.38
Customer 804 initiated a call at 4990.06
Customer 803 A caller hangs up -error- at 4990.79
Customer 801 is done operator2 at 4994.44
Customer 804 is assigned to a operator2 at 4995.6
Customer 804 is done operator2 at 4997.9
Customer 798 is assigned to a operator2 at 4997.9
Customer 805 initiated a call at 5002.06
Customer 798 is done operator2 at 5005.6
Operator2 gives break at 5006.5
Operator2 break done at 5009.5
break2 67
Customer 805 is assigned to a operator2 at 5010.69
Customer 805 is done operator2 at 5012.28
Customer 800 is done operator1 at 5013.04
Cus

Operator2 break done at 1258.74
break2 21
Customer 222 initiated a call at 1262.04
Customer 222 A caller hangs up -error- at 1271.94
Customer 223 initiated a call at 1276.84
Customer 224 initiated a call at 1281.57
Customer 225 initiated a call at 1292.11
Customer 226 initiated a call at 1300.77
Customer 227 initiated a call at 1302.4
Customer 227 is assigned to a operator2 at 1304.82
Customer 227 is done operator2 at 1306.12
Customer 226 is assigned to a operator2 at 1308.3
Customer 228 initiated a call at 1309.6
Customer 226 is done operator2 at 1313.99
Customer 228 is assigned to a operator2 at 1320.18
Customer 228 is done operator2 at 1324.67
Customer 229 initiated a call at 1328.35
Customer 229 A caller hangs up -error- at 1329.76
Customer 230 initiated a call at 1336.13
Customer 230 is assigned to a operator2 at 1337.82
Customer 230 is done operator2 at 1342.03
Customer 231 initiated a call at 1350.92
Customer 231 is assigned to a operator2 at 1355.25
Customer 231 is done operato

KeyboardInterrupt: 

In [ ]:
#1k

#utilization average of operator 1
utilOf1KOP1=sumUtil1KOP1/10

#utilization average of operator 2
utilOf1KOP2=sumUtil1KOP2/10

#average of waiting times 
avg1kWaitingTimes=sumOf1kWaitingTimes/10

#average of operator1 waiting times
avgof1kOp1wTimes=sumof1kOp1wTimes/10

#average of operator2 waiting times
avgof1kOp2wTimes=sumof1kOp2wTimes/10

#average of Unsatisfied Customer leaving times
avgof1kUnwTimes=sumof1kUnwTimes/10

#average of Util time answering system

avgof1kUtilAnswering=sumof1kUtilAnswering/(10*100)




print('1K AVERAGE STATISTICS')

print('Utilization of The answering System %g' % avgof1kUtilAnswering)
print('Utilization of operator1 %g' % utilOf1KOP1)
print('Utilization of operator1 %g' % utilOf1KOP1)
print('Utilization of operator2 %g' % utilOf1KOP2)
print('Average waiting times of customers %g' % avg1kWaitingTimes)
print('Maximum Total Waiting Time to Total System Time Ratio %g' % maxRatio1K )
print('Average number of people waiting to be served by operator1. %g' % avgof1kOp1wTimes )
print('Average number of people waiting to be served by operator2. %g' % avgof1kOp2wTimes )
print('Average number of Unsatisfied Customer leaving. %g' % avgof1kUnwTimes )





In [ ]:
CUSTOMER_COUNT=5000
for i in range(10):
    RANDOM_SEED+=1
    env = simpy.Environment()
    operator1 = simpy.Resource(env, capacity = 1)
    operator2 = simpy.Resource(env, capacity = 1)
    env.process(customer_generator(env, operator1, operator2))
    env.process(break_generator1(env, operator1))
    env.process(break_generator2(env, operator2))
    env.run(until=32000)
    
     #statistic calculations
    
    
    
    
    #utilization of operators
    serviceTimeOfOP1=sum(service_times_operator1)
    utilizaztionOfOP1=serviceTimeOfOP1/end_time
    sumUtil5KOP1+=utilizaztionOfOP1
    serviceTimeOfOP2=sum(service_times_operator2)
    utilizaztionOfOP2=serviceTimeOfOP2/end_time
    sumUtil5KOP2+=utilizaztionOfOP2
    
    #total waiting time
    sumOf5kWaitingTimes+=sum(queue_waiting)
    
    #Max Ratio
    maxRatio5K=max(sum(queue_waiting)/end_time,maxRatio5K)
    
    #Sum of waiting time of operator1
    sumof5kOp1wTimes+=sum(queue_w_op1)
    
    #Sum of waiting time of operator2
    sumof5kOp2wTimes+=sum(queue_w_op2)
    
    #Sum of Unsatisfied costumer leaving times
    sumof5kUnwTimes+=(sum(unsatisfied_customer))
    
    #Sum of Util time answering system

    sumof5kUtilAnswering+=(sum(answering_util)/end_time)
    
    
    #reset variables
    service_times_operator1 = []

    service_times_operator2 = []

    queue_waiting = []
    
    queue_w_op1 = []
    
    queue_w_op2 = []

    queue_w_customer = []

    queue_w_operator1 = []

    queue_w_operator2 = []
    
    end_time = 0
    
    unsatisfied_customer=[]
    
    answering_util= []

In [ ]:
#5k

#utilization average of operator 1
utilOf5KOP1=sumUtil5KOP1/10

#utilization average of operator 2
utilOf5KOP2=sumUtil5KOP2/10

#average of waiting times 
avg5kWaitingTimes=sumOf5kWaitingTimes/10

#average of operator1 waiting times
avgof5kOp1wTimes=sumof5kOp1wTimes/10

#average of operator2 waiting times
avgof5kOp2wTimes=sumof5kOp2wTimes/10

#average of Unsatisfied Customer leaving times
avgof5kUnwTimes=sumof5kUnwTimes/10

#average of Util time answering system
avgof5kUtilAnswering=sumof5kUtilAnswering/(10*100)

print('5K AVERAGE STATISTICS')

print('Utilization of the answering System %g' % avgof5kUtilAnswering)
print('Utilization of operator1 %g' % utilOf5KOP1)
print('Utilization of operator2 %g' % utilOf5KOP2)
print('Average waiting times of customers %g' % avg5kWaitingTimes)
print('Maximum Total Waiting Time to Total System Time Ratio %g' % maxRatio5K )
print('Average number of people waiting to be served by operator1. %g' % avgof5kOp1wTimes )
print('Average number of people waiting to be served by operator2. %g' % avgof5kOp2wTimes )
print('Average number of Unsatisfied Customer leaving. %g' % avgof5kUnwTimes )


